In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.datasets.fashion_mnist import load_data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# Przykładowy obraz ze zbioru treningowego

In [ ]:
X_train[0]

In [ ]:
# To samo, tylko jako faktyczny obrazek

In [ ]:
plt.figure()
plt.imshow(X_train[0], cmap='Greys')
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Klasa dla obrazu powyżej  

In [ ]:
y_train[0]

In [ ]:
# Jeszcze jeden przykład

In [ ]:
plt.imshow(X_train[10], cmap='Greys')
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
y_train[10]

In [ ]:
# https://github.com/zalandoresearch/fashion-mnist

mapa_kategorii = {
    0 : 'T-shirt/top',
    1 : 'Trouser',
    2 : 'Pullover',
    3 : 'Dress',
    4 : 'Coat',
    5 : 'Sandal',
    6 : 'Shirt',
    7 : 'Sneaker',
    8 : 'Bag',
    9 : 'Ankle boot'
}

In [ ]:
mapa_kategorii[y_train[10]]

In [ ]:
# Skalowanie i normalizacja

In [ ]:
X_train = X_train/255.0

In [ ]:
y_train = y_train.astype(np.int32) 

In [ ]:
# Budowanie modelu

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    metrics=['accuracy']
)

In [ ]:
# Szkolenie przez 2 epoki, tj dwukrotne przejście przez zbiór uczący

In [ ]:
model.fit(X_train, y_train, epochs=2)

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

In [ ]:
X_test = X_test/255.0

In [ ]:
y_test = y_test.astype(np.int32)

In [ ]:
# Przewidywanie zbioru testowego

In [ ]:
y_pred = model.predict(X_test, steps = 10)
print(y_pred.shape)

In [ ]:
y_pred[0]

In [ ]:
# Predyklcja zawiera wektor - prawdopodobieństwo przynależności do każdej z klas. Sprawdźmy która klasa ma największe prawdopodobieństwo.

In [ ]:
np.argmax(y_pred[0])

In [ ]:
mapa_kategorii[np.argmax(y_pred[0])]

In [ ]:
mapa_kategorii[y_test[0]]

In [ ]:
# Pierwszy element to (u mnie) półbut. Czy tak jest naprawdę?

In [ ]:
plt.imshow(X_test[0], cmap='Greys')
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Sprawdźmy jeszcze prawdopodobieństwa dla predykcji

In [ ]:
plt.grid(False)
plt.xticks(range(10))
plt.yticks([])
plt.bar(range(10), y_pred[0])
plt.ylim([0, 1])

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

In [ ]:
# Dodatek 1: Mapy aktywacji

In [ ]:
nr_kategorii = 0
mapa_aktywacji = []
for elem in model.weights[0].numpy():
    mapa_aktywacji.append(elem[nr_kategorii])
    
plt.imshow(np.array(mapa_aktywacji).reshape((28, 28)))

In [ ]:
# Dodatek 2: Własny przykład

In [ ]:
def decode_img(filepath):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [28, 28])

In [ ]:
lista_plikow = tf.data.Dataset.list_files(str('C:/Users/Adam/Desktop/koszula*'))

In [ ]:
moj_test = lista_plikow.map(decode_img)

In [ ]:
obraz = None
for elem in moj_test.take(1):
    obraz = elem.numpy()

plt.imshow(obraz.reshape(28, 28), cmap='Greys')
plt.show()

In [ ]:
y_pred_moj = model.predict(obraz.reshape(1, 784))

In [ ]:
y_pred_moj

In [ ]:
mapa_kategorii[np.argmax(y_pred_moj)]

In [ ]:
plt.grid(False)
plt.xticks(range(10))
plt.yticks([])
plt.bar(range(10), y_pred_moj[0])
plt.ylim([0, 1])

In [ ]:
# Zadanie: zrób lub znajdź zdjęcie ubioru, wczytaj jak powyżej a następnie dokonaj predykcji na tym zdjęciu

Dodatek 3:

Przykładowa prosta sieć konwolucyjna.

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model2.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    metrics=['accuracy']
)


In [ ]:
model2.fit(X_train.reshape(60000, 28, 28, 1), y_train, epochs=2)

In [ ]:
for weight in model2.weights:
    print(weight.shape)